In [1]:
import os
import glob
import json
import shutil

# Example code

In [2]:
DATASET_DIRECTORY = os.path.join(os.getcwd(), 'ADE20K_2021_17_01', 'images', 'ADE', 'training', 'home_or_hotel')
SUB_DIRECTORIES = next(os.walk(DATASET_DIRECTORY))[1]

## Filtering only floor images respective masks from home_or_hotel data 

In [5]:
for directory in SUB_DIRECTORIES:

    new_directory = os.path.join(DATASET_DIRECTORY, directory).split(os.sep)
    new_directory= [temp.replace('ADE20K_2021_17_01', 'ADE20K_FloorData')  for temp in new_directory]
    new_directory = os.sep.join(new_directory)

    if not os.path.exists(new_directory):
        os.makedirs(new_directory)
        os.makedirs(os.path.join(new_directory,'rgb'))
        os.makedirs(os.path.join(new_directory,'mask'))

    RBG_IMAGES = glob.glob(os.path.join(DATASET_DIRECTORY, directory, '*.jpg'))
    JSON_FILES = glob.glob(os.path.join(DATASET_DIRECTORY, directory, '*.json'))

    mask_images=[]
    for file in JSON_FILES:
        with open(file, 'r') as f:
            parsed = json.load(f)
            objects = parsed['annotation']['object']
            floor_objects = [obj for obj in objects if obj["raw_name"]=="floor"]

        if len(floor_objects)==1:
            for obj in floor_objects:
                mask_images.append(obj['instance_mask'].split('/'))

    MASK_IMAGES = [os.path.join(DATASET_DIRECTORY, directory, image[0], image[1]) for image in mask_images]
    to_be_transfered = [temp.split(os.sep)[-2] for temp in MASK_IMAGES]
 
    for file in RBG_IMAGES:
        if file.split(os.sep)[-1].split('.')[0] in to_be_transfered:
            shutil.copy(file, os.path.join(new_directory,'rgb'))
    
    for file in MASK_IMAGES:
        if file.split(os.sep)[-2] in to_be_transfered:
            shutil.copy(file, os.path.join(new_directory,'mask'))
            dst_file = os.path.join(os.path.join(new_directory,'mask'), file.split(os.sep)[-1])
            new_dst_file_name = os.path.join(os.path.join(new_directory,'mask'), file.split(os.sep)[-2]+'.png')
            os.rename(dst_file, new_dst_file_name)